In [51]:
import numpy as np

from keras.datasets import imdb

from keras.preprocessing.sequence import pad_sequences

from keras import models
from keras.layers import Dense
#from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D

## Téléchargement des données

In [52]:
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=12000)
#X_train

## padder

In [3]:
v1 = np.array([1, 2, 3, 4])
v2 = np.array([11, 222])
pad_sequences([v1, v2])

array([[  1,   2,   3,   4],
       [  0,   0,  11, 222]])

In [53]:
pad_sequences(X_train)
print(X_train.shape)
pad_train =  pad_sequences(X_train, maxlen = 80)
print(pad_train)
print(pad_train.shape)
pad_test = pad_sequences(X_test, maxlen = 80)
print(pad_test.shape)

(25000,)
[[  15  256    4 ...   19  178   32]
 [ 125   68    2 ...   16  145   95]
 [ 645  662    8 ...    7  129  113]
 ...
 [ 529  443    2 ...    4 3586    2]
 [ 286 1814   23 ...   12    9   23]
 [  97   90   35 ...  204  131    9]]
(25000, 80)
(25000, 80)


## Le modèle
### composé d'une première couche Embeding suivi d'un lstm terminer d'une dense
Cette couche est chargée transformer vos documents en un vecteur de taille beaucoup plus petite que si ovus aviez utilisé le one-hot encoding

### Fit

In [54]:
model = models.Sequential()
model.add(Embedding(100000, 32))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, None, 32)          3200000   
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               82432     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 3,282,561
Trainable params: 3,282,561
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

entr = model.fit(pad_train, Y_train,
                 batch_size = 64, 
                 epochs = 10, 
                 validation_split = 0.2
         )

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 63s 3ms/step - loss: 0.5096 - acc: 0.7468 - val_loss: 0.3858 - val_acc: 0.8302
Epoch 2/10
20000/20000 [==============================] - 65s 3ms/step - loss: 0.3320 - acc: 0.8644 - val_loss: 0.3920 - val_acc: 0.8264
Epoch 3/10
20000/20000 [==============================] - 57s 3ms/step - loss: 0.2798 - acc: 0.8888 - val_loss: 0.4019 - val_acc: 0.8180
Epoch 4/10
20000/20000 [==============================] - 57s 3ms/step - loss: 0.2361 - acc: 0.9100 - val_loss: 0.4258 - val_acc: 0.8104
Epoch 5/10
20000/20000 [==============================] - 58s 3ms/step - loss: 0.2131 - acc: 0.9194 - val_loss: 0.4505 - val_acc: 0.81982119 - a
Epoch 6/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.1827 - acc: 0.9330 - val_loss: 0.5050 - val_acc: 0.8140
Epoch 7/10
20000/20000 [==============================] - 66s 3ms/step - loss: 0.1525 - acc: 0.9454 - val_loss: 0.5398 - 

In [56]:
print(model.evaluate(pad_train, Y_train))
print()
print(model.evaluate(pad_test, Y_test))

model.predict(pad_test)

25000/25000 [==============================] - 14s 563us/step
[0.17324588351130485, 0.95172]

25000/25000 [==============================] - 16s 641us/step
[0.6511789025592803, 0.8076]


array([[0.9873806 ],
       [0.99946445],
       [0.87361825],
       ...,
       [0.01520179],
       [0.0014717 ],
       [0.99354035]], dtype=float32)

### Convolution 1D

In [57]:
model1D = models.Sequential()
model1D.add(Embedding(100000, 32))
model1D.add(Conv1D(16, 3, activation='relu'))
model1D.add(GlobalAveragePooling1D())
model1D.add(Dense(1, activation='sigmoid'))

print(model1D.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, None, 32)          3200000   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, None, 16)          1552      
_________________________________________________________________
global_average_pooling1d_10  (None, 16)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 3,201,569
Trainable params: 3,201,569
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
model1D.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model1D.fit(pad_train, Y_train,
                 batch_size = 64, 
                 epochs = 10, 
                 validation_split = 0.2
         )

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 21s 1ms/step - loss: 0.5577 - acc: 0.7504 - val_loss: 0.3991 - val_acc: 0.8240
Epoch 2/10
20000/20000 [==============================] - 20s 996us/step - loss: 0.3256 - acc: 0.8640 - val_loss: 0.3602 - val_acc: 0.8396
Epoch 3/10
20000/20000 [==============================] - 20s 1ms/step - loss: 0.2558 - acc: 0.8971 - val_loss: 0.3629 - val_acc: 0.8378
Epoch 4/10
20000/20000 [==============================] - 20s 997us/step - loss: 0.2122 - acc: 0.9193 - val_loss: 0.3845 - val_acc: 0.8334
Epoch 5/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.1806 - acc: 0.9344 - val_loss: 0.4157 - val_acc: 0.8288
Epoch 6/10
20000/20000 [==============================] - 19s 956us/step - loss: 0.1543 - acc: 0.9451 - val_loss: 0.4574 - val_acc: 0.8212
Epoch 7/10
20000/20000 [==============================] - 19s 966us/step - loss: 0.1328 - acc: 0.9546 - val_loss: 0.5000 - 

### 